## Screening the Materials Project for Previously Unsynthesized Materials 
## For trial synthesis in the A-Lab 
 
**Authors**: Matthew McDermott and Max Gallant

**Date**: Last updated July 24, 2023

#### Suggested filters 
Some of these were suggested by Nathan.

- Not previously known to be synthesized (i.e., not in ICSD experimental DB or experimental literature)
- Thermodynamically stable (i.e. <0.001 eV/atom $E_{hull}$)
- Reasonable oxidation states (e.g., Mn3+, not Mn7+)
- Must be of a common (and easy-to-handle) composition type, including but not limited to: oxides, sulfates, phosphates, silicates, fluorides, chlorides, bromides, borates
- Exclude all expensive elements (Rh, Ir, etc.)
- Exclude all dangerous elements (radioactive elements like Pu or toxic elements like As)

#### Imports

In [1]:
from collections import OrderedDict
import re

import emmet
from emmet.core.summary import HasProps
import matplotlib.pyplot as plt
from monty.serialization import dumpfn, loadfn
from mp_api.client import MPRester
import pandas 
from pymatgen.core.composition import Composition, Element
import seaborn as sns

sns.set("talk",style="ticks")

%load_ext autoreload
%autoreload 2

In [2]:
radioactive = ["Ac","Th","Pa","U","Np","Pu","Tc"]  # all other radioactive elements are not on MP
expensive = ["Cs","Rb","Pm","Tm","Sc","Tl","Re","Ir","Os","Pd","Ru","Pt","Rh","Au"]
dangerous = ["Hg","As"]

exclude_elements = radioactive + expensive + dangerous

#### Get summary documents with MPRester

In [3]:
with MPRester() as mpr:
    materials = mpr.summary.search(theoretical=True, 
                                   energy_above_hull=(0.000,0.001), 
                                   exclude_elements=exclude_elements, 
                                   all_fields=False,
                                   deprecated=False,
                                   has_props=[HasProps("oxi_states")],
                                   fields=["material_id","composition","formation_energy_per_atom", 
                                          "energy_above_hull", "equilibrium_reaction_energy_per_atom"])

Retrieving SummaryDoc documents:   0%|          | 0/6522 [00:00<?, ?it/s]

#### Get oxidation state docs with MPRester

In [4]:
with MPRester() as mpr:
    oxi_states = mpr.oxidation_states.search([doc.material_id for doc in materials], fields=["material_id","average_oxidation_states", "structure"])

Retrieving OxidationStateDoc documents:   0%|          | 0/6522 [00:00<?, ?it/s]

#### Join and filter by compounds with pre-computed oxidation states
Note: we will lose some compounds that don't have oxidation states calculated (this is okay).

In [5]:
all_data = []
for d1 in materials:
    d2 = list(filter(lambda i: i.material_id == d1.material_id, oxi_states))[0]
    average_oxidation_states = d2.average_oxidation_states
    if average_oxidation_states:
        doc = d1.dict()
        del doc["fields_not_requested"]
        doc["average_oxidation_states"] = average_oxidation_states
        doc["structure"] = d2.structure
        all_data.append(doc)

In [6]:
len(all_data)  # this number will be lower than the original number

4723

#### Removing any duplicate formulas
There seem to be a few compounds with multiple stable (or close-to-stable entries).

In [7]:
seen_formulas = set()
all_data_no_duplicates = []

for d in sorted(all_data, key=lambda i: i["formation_energy_per_atom"]):
    formula = d["composition"].reduced_formula 
    if formula in seen_formulas:
        continue
    else:
        all_data_no_duplicates.append(d)
        seen_formulas.add(formula)

In [8]:
len(all_data_no_duplicates)

4548

#### Filter by unreasonable oxidation states
If the average oxidation state is outside of the range spanned by the `common_oxidation_states` property (see pymatgen), then we will discard.

For convenience, we will print out all the discard compositions and their avg. oxidation states.

In [9]:
filtered_data = []
for d in all_data_no_duplicates:
    species = set(d["structure"].species)
    bad = False
    for specie in species:
        common_oxidation_states = specie.element.common_oxidation_states
        if int(specie.oxi_state) not in range(min(common_oxidation_states), max(common_oxidation_states)+1):
            print(specie, d["composition"])
            bad=True
            break
    if not bad:
        filtered_data.append(d)

Yb2+ Yb2 F4
Yb2+ Yb8 Al8 F40
Yb2+ Yb4 Hf4 O12
Ag2+ Ca6 Hf6 Ag2 F40
Yb2+ Yb4 Ti4 O12
V3+ Ba4 Mg4 V4 F28
V3+ Ca2 V2 F10
V3+ Ba4 Ca2 Mn2 V4 F28
Ti3.75+ Ca3 Y1 Ti4 O12
Cr2+ Zr1 Cr1 F6
Ti3+ Sr4 La1 Ti5 O15
Pr2+ Ca2 Pr2 Ti4 O12
V3+ Ba4 Ca2 V4 Fe2 F28
Ti3+ Ca18 La2 Ti20 O60
Ti3+ Sr18 La2 Ti20 O60
V3+ Sr2 Li2 V2 F12
V3+ Li2 Ca2 V2 F12
V3+ Na4 Sr4 V4 F24
Cr2+ Ca2 Cr2 F8
V3+ Ba8 Na8 V8 F48
V4+ Sr8 V8 O4 F40
Ce2+ Ce22 Mn2 S32
Mo3+ Ba4 Mg4 Mo4 F28
Ti3+ Ba2 Ti16 O32
Ti3+ Ba6 Ta9 Ti7 O42
Ti3+ Ba3 Ti20 O40
Ag2+ K6 Hf8 Ag4 F46
V4+ Ca1 V1 F6
Ti3+ Mg11 Ti25 O60
Ti3+ Mg5 Ti13 O30
Ti3+ Mg3 Ti9 O20
Ti3+ Mg4 Ti14 O30
Ti3+ Mg2 Ti10 O20
Nd2+ Nd4 Eu8 Be4 O20
Ti3.83333333+ K2 Ti12 O24
Nb4+ Ba6 Ta8 Nb2 O30
V4+ La2 Mg1 V1 O6
Ti3+ Na4 Ti20 O40
V4+ Ba4 Ca1 V4 Cu2 F28
V3+ Ba4 V4 Zn4 F28
V3+ Na4 Mg2 V2 F14
V3+ Ba2 V2 Cu1 F12
Nb4+ Ba6 Ti7 Nb9 O42
Yb2+ Yb2 Mg1 Nb2 O8
Ti3+ K2 Na1 Ti1 F6
Ti3+ K6 Ti16 O34
V3+ Ba4 V4 Cd4 F28
V3+ Sr8 V4 C4 O12 F20
Mo3+ Ba4 Ca2 Cu2 Mo4 F28
V3+ Sr4 Ta2 V2 O12
Ti3+ Na6 Ti2 F12
V3+ Ca4 Ta2 V2 O

In [10]:
Element("Fe").common_oxidation_states  # for example, no Fe4+

(2, 3)

In [11]:
filtered_data_df = []
for d in filtered_data:
    clean_d = {"formula":d["composition"].reduced_formula}
    clean_d["mpid"] = str(d["material_id"])
    clean_d["e_hull"] = d["energy_above_hull"]
    clean_d["e_f"] = round(d["formation_energy_per_atom"],4)
    clean_d["e_d"] = round(d["equilibrium_reaction_energy_per_atom"],4) if d["equilibrium_reaction_energy_per_atom"] else None
    clean_d["avg_oxi"] = d["average_oxidation_states"]
    clean_d["num_elems"] = len(d["composition"].elements)
    filtered_data_df.append(clean_d)
    
df = pandas.DataFrame(filtered_data_df)

In [12]:
len(filtered_data_df)

3372

In [13]:
df.sort_values("e_d").head()  # looking good!

,formula,mpid,e_hull,e_f,e_d,avg_oxi,num_elems
1,DyF3,mp-1212843,0.0,-4.4680,-4.4680,"{'Dy': 3.0, 'F': -1.0}",2
2,LuF3,mp-1207204,0.0,-4.4534,-4.4534,"{'Lu': 3.0, 'F': -1.0}",2
53,GdF3,mp-972965,0.0,-4.1128,-4.1128,"{'Gd': 3.0, 'F': -1.0}",2
231,BeF2,mp-561543,0.0,-3.5527,-3.5527,"{'Be': 2.0, 'F': -1.0}",2
412,Ta2O5,mp-1539317,0.0,-3.3365,-3.3365,"{'Ta': 5.0, 'O': -2.0}",2


#### Removing compounds which have other synthesized polymorphs

Materials Project sometimes gets the incorrect polymorph ordering for several compounds.

For example, MP may indicate that our downloaded entry for LiBr has never been synthesized. While it is true that that particular polymorph may have never been synthesized, there is another one that HAS been synthesized and calculated to be metastable (it could even be just +0.002 eV/atom above the hull and therefore not found in our search).

Therefore we'd like to remove compounds that are associated with *any* previously synthesized crystal structure.

In [14]:
with MPRester() as mpr:
    previously_synthesized = mpr.summary.search(theoretical=False, 
                                   exclude_elements=exclude_elements, 
                                   all_fields=False,
                                   deprecated=False,
                                   has_props=[emmet.core.summary.HasProps("oxi_states")],
                                  fields=["composition"])

Retrieving SummaryDoc documents:   0%|          | 0/32545 [00:00<?, ?it/s]

In [15]:
known_formulas = {d.composition.reduced_formula for d in previously_synthesized}

No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.


In [16]:
filtered_data_df_2 = [d for d in filtered_data_df if d['formula'] not in known_formulas]

In [17]:
df = pandas.DataFrame(filtered_data_df_2)

In [18]:
len(df)

3045

#### Removing binary compounds
Many of these already exist. As we are looking for purely novel compounds, it's best to get rid of these.

In [19]:
df_2 = df[df["num_elems"]>2]
df_2

,formula,mpid,e_hull,e_f,e_d,avg_oxi,num_elems
0,BaGdCrFeO6,mp-1518724,0.000000,-5.1543,-2.4397,"{'Ba': 2.0, 'Gd': 3.0, 'Cr': 5.0, 'Fe': 2.0, '...",5
3,BaLu2F8,mp-1214962,0.000000,-4.4284,-0.0539,"{'Ba': 2.0, 'Lu': 3.0, 'F': -1.0}",3
4,BaY2F8,mp-768240,0.000000,-4.3848,-0.0106,"{'Ba': 2.0, 'Y': 3.0, 'F': -1.0}",3
5,Sr4Lu3F17,mp-35769,0.000000,-4.3724,-0.0294,"{'Sr': 2.0, 'Lu': 3.0, 'F': -1.0}",3
6,LuOF,mp-1210775,0.000000,-4.3444,-0.0331,"{'Lu': 3.0, 'O': -2.0, 'F': -1.0}",3
...,...,...,...,...,...,...,...
3035,KCuH3,mp-1147650,0.000000,-0.1126,-0.0015,"{'K': 1.0, 'Cu': 2.0, 'H': -1.0}",3
3036,MoWC2,mp-1221393,0.000801,-0.1023,NaN,"{'Mo': 4.0, 'W': 4.0, 'C': -4.0}",3
3038,Al3BC3,mp-1228599,0.000000,-0.0922,-0.0058,"{'Al': 3.0, 'B': 3.0, 'C': -4.0}",3
3040,Mg(AlGe)2,mp-1232292,0.000000,-0.0827,-0.0020,"{'Mg': 2.0, 'Al': 3.0, 'Ge': -4.0}",3


#### Filter by literature data
The Materials Project uses a reference to the ICSD database to check if a compound has been synthesized. Because this may be outdated, we now turn to the recent Kononova et al. literature dataset to check for any compounds that have been experimentally synthesized.

Here search the sol-gel and solid-state datasets by formula and filter out any compounds that appear in either.

In [20]:
f1 = loadfn("solid-state_dataset_20200713.json.xz")
f2 = loadfn("sol-gel_dataset_20200713.json.xz")

In [21]:
def get_formulas_from_lit(f):
    all_formulas = set()
    for r in f["reactions"]:
        for precursor in r["precursors"]:
            try:
                all_formulas.add(Composition(precursor["material_formula"]).reduced_formula)
            except Exception as e:
                pass
        try:
            all_formulas.add(Composition(r["target"]["material_formula"]).reduced_formula)
        except Exception as e:
            pass
                
    return all_formulas

In [22]:
all_lit_formulas = get_formulas_from_lit(f1) | get_formulas_from_lit(f2)

In [23]:
df_3 = df_2[[f not in all_lit_formulas for f in df_2["formula"]]]

In [24]:
len(df_3)

2759

#### Label common composition types
The goal here is to prioritize synthesizing compounds of desired formula types (and avoiding others). 

*We must define these first since they don't seem to be pre-defined in pymatgen...!*

In [25]:
def get_formula_type_by_anion(formula):
    polyatomics={
        'PO3': 'phosphite',
        'PO4': 'phosphate',
        'HPO4': 'hydrogen phosphate',
        'H2PO4': 'dihydrogen phosphate',
        'BO3': 'orthoborate',
        'BO2': 'metaborate',
        'B2O5': 'diborate',
        'B3O7': 'triborate',
        'B4O7': 'tetraborate',
        'ClO4': 'perchlorate',
        'ClO3': 'chlorate',
        'ClO2': 'chlorite',
        'ClO': 'hypochlorite',
        'OCl': 'oxychloride',
        'BrO4': 'perbromate',
        'BrO3': 'bromate',
        'BrO2': 'bromite',
        'BrO': 'hypobromite',
        'IO4': 'periodate',
        'IO3': 'iodate',
        'IO2': 'iodite',
        'IO': 'hypoiodite',
        'NH4': 'ammonium',
        'NO2': 'nitrite',
        'NO3': 'nitrate',
        'SO3': 'sulfite',
        'SO4': 'sulfate',
        'HSO3': 'bisulfite',
        'HSO4': 'bisulfate',
        'S2O3': 'thiosulfate',
        'HS2O3': 'hydrogen thiosulfate',
        'C2O4': 'oxalate',
        'HC2O4': 'hydrogen oxalate',
        'OH': 'hydroxide',
        'CH3COO': 'acetate',
        'CO3': 'carbonate',
        'HCO3': 'bicarbonate',
        'CrO4': 'chromate',
        'Cr2O7': 'dichromate',
        'MnO4': 'permanganate',
        'CN': 'cyanide',
        'OCN': 'cyanate',
        'SCN': 'thiocyanate',
        'C8H4O4': 'phthalate',
        'C2H3O2': 'acetate',
        'HCO2': 'fomate',
        'SiO4': 'silicate'
    }
    polyatomics = OrderedDict({k: polyatomics[k] for k in sorted(polyatomics, key=len, reverse=True)})
    anions = {"H":'hydride',
            "F":"fluoride",
              "Cl":"chloride",
              "Br":"bromide",
              "I":"iodide",
              "O":"oxide",
              "S":"sulfide",
              "Se":"selenide",
              "Te":"telluride",
              "N":"nitride", 
              "P":"phosphide",
              "As":"arsenide",
              "Sb":"antimonide",
              "Bi":"bismide",
              "C":"carbide",
              "Si":"silicide",
              "Ge":"germanide",
             "Sn":"stannide",
             "Pb":"plumbide",
             "B":"boride",
             "Al":"aluminide",
             "Ga":"gallide",
             "In":"indide",
             "Tl":"thallide",
             "Ni":"nickelide",
             "Mo":"molybdenide"}
    
    for p, name in polyatomics.items():
        if p in formula:
            return name
    
    return anions[re.findall(r"([A-Z][a-z]?)(\d*\)?\d*$)", formula)[0][0]]

In [26]:
df_3["type"] = [get_formula_type_by_anion(f) for f in df_3["formula"]]


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Desired composition types:

In [27]:
desired = ["oxide","sulfite","sulfate","bisulfate","silicate",
           "fluoride","chloride","bromide","orthoborate", "metaborate",
           "hydroxide","carbonate","tetraborate","phosphate","phosphite",
           "chlorate","chlorite","hypochlorite","oxychloride","bicarbonate",
          ]

In [28]:
df_4 = df_3[[r["type"] in desired for _, r in df_3.iterrows()]]

In [29]:
df_4.sort_values("e_d").head(10)

,formula,mpid,e_hull,e_f,e_d,avg_oxi,num_elems,type
0,BaGdCrFeO6,mp-1518724,0.0,-5.1543,-2.4397,"{'Ba': 2.0, 'Gd': 3.0, 'Cr': 5.0, 'Fe': 2.0, '...",5,oxide
1909,Na3ErBr6,mp-1210529,0.0,-2.0638,-0.9318,"{'Na': 1.0, 'Er': 3.0, 'Br': -1.0}",3,bromide
1921,Li3ErBr6,mp-1222492,0.0,-2.0536,-0.9269,"{'Li': 1.0, 'Er': 3.0, 'Br': -1.0}",3,bromide
1166,Tb2(SO4)3,mp-1208801,0.0,-2.6597,-0.3736,"{'Tb': 3.0, 'S': 6.0, 'O': -2.0}",3,sulfate
1212,Sm2(SO4)3,mp-1208980,0.0,-2.6233,-0.3730,"{'Sm': 3.0, 'S': 6.0, 'O': -2.0}",3,sulfate
1156,Dy2(SO4)3,mp-1213482,0.0,-2.6635,-0.3698,"{'Dy': 3.0, 'S': 6.0, 'O': -2.0}",3,sulfate
1144,Ho2(SO4)3,mp-768978,0.0,-2.6695,-0.3688,"{'Ho': 3.0, 'S': 6.0, 'O': -2.0}",3,sulfate
1116,Lu2(SO4)3,mp-768453,0.0,-2.6833,-0.3559,"{'Lu': 3.0, 'S': 6.0, 'O': -2.0}",3,sulfate
527,La5Mn5O16,mp-531717,0.0,-3.1633,-0.3022,"{'La': 3.0, 'Mn': 3.4, 'O': -2.0}",3,oxide
965,YbCO3,mp-755213,0.0,-2.8150,-0.3007,"{'Yb': 3.0, 'C': 3.0, 'O': -2.0}",3,carbonate


In [30]:
df_4.value_counts("type")  # most are oxides!

type
oxide           1241
fluoride         293
phosphate        115
chloride          77
sulfate           72
bromide           41
phosphite         27
carbonate         23
orthoborate       21
hypochlorite      19
chlorate          10
silicate           9
chlorite           8
metaborate         8
sulfite            4
Name: count, dtype: int64

In [31]:
len(df_4)

1968

#### Removing any missed ICSD compounds
Citrine Informatics has available a list of ICSD formulas found here: https://github.com/CitrineInformatics/learn-citrination/blob/master/citrination_api_examples/tutorial_sequence/example_data/icsd.dat 

Even though we previously filtered out compounds that the Materials Project said were in ICSD, this updated list will catch some ones that are more recent / missing from MP.

In [32]:
citrine = loadfn("icsd_compounds.json.gz")
df_5 = df_4[[f not in citrine for f in df_4["formula"]]]

In [33]:
len(df_5)

1448

#### Removing compounds that appear in De Gruyter's Handbook of Inorganic Substances (2017)
Handbook of Inorganic Substances (2017): https://www.degruyter.com/document/doi/10.1515/9783110445404/html?lang=en

Using Optical Character Recognition, we scraped the formulas from this handbook. This seems to be a much more encompassing source (often containing compounds that cannot even be identified from Google searches.)

In [35]:
de_gruyter = loadfn("de_gruyter_formulas.json.gz")
de_gruyter = set(de_gruyter)

In [36]:
len(de_gruyter)

99842

In [37]:
df_6 = df_5[[f not in de_gruyter for f in df_5["formula"]]]

In [38]:
len(df_6)

616

In [39]:
df_6.sort_values(["e_d", "e_hull"]).head(15)

,formula,mpid,e_hull,e_f,e_d,avg_oxi,num_elems,type
0,BaGdCrFeO6,mp-1518724,0.0,-5.1543,-2.4397,"{'Ba': 2.0, 'Gd': 3.0, 'Cr': 5.0, 'Fe': 2.0, '...",5,oxide
527,La5Mn5O16,mp-531717,0.0,-3.1633,-0.3022,"{'La': 3.0, 'Mn': 3.4, 'O': -2.0}",3,oxide
381,Eu5P3O13,mp-1212818,0.0,-3.3259,-0.2512,"{'Eu': 2.2, 'P': 5.0, 'O': -2.0}",3,oxide
2161,Eu2PCl,mp-1225319,0.0,-1.7790,-0.2175,"{'Eu': 2.0, 'P': -3.0, 'Cl': -1.0}",3,chloride
47,YbGd3F12,mp-1215594,0.0,-4.1427,-0.2041,"{'Yb': 3.0, 'Gd': 3.0, 'F': -1.0}",3,fluoride
111,BaCeEuHfO6,mp-1516655,0.0,-3.8470,-0.1453,"{'Ba': 2.0, 'Ce': 4.0, 'Eu': 2.0, 'Hf': 4.0, '...",5,oxide
349,CaWF6,mp-1395540,0.0,-3.3691,-0.1426,"{'Ca': 2.0, 'W': 4.0, 'F': -1.0}",3,fluoride
1727,HfZn(SO4)3,mp-2713876,0.0,-2.2481,-0.1379,"{'Hf': 4.0, 'Zn': 2.0, 'S': 6.0, 'O': -2.0}",4,sulfate
482,MgWF6,mp-1407403,0.0,-3.2070,-0.1286,"{'Mg': 2.0, 'W': 4.0, 'F': -1.0}",3,fluoride
904,Hf2Si(SO6)2,mp-2713341,0.0,-2.8616,-0.1188,"{'Hf': 4.0, 'Si': 4.0, 'S': 6.0, 'O': -2.0}",4,oxide


#### Saving as Excel file

In [40]:
df_6.sort_values(["e_d", "e_hull"]).to_excel("screened_targets.xlsx")

In [41]:
df_6

,formula,mpid,e_hull,e_f,e_d,avg_oxi,num_elems,type
0,BaGdCrFeO6,mp-1518724,0.000000,-5.1543,-2.4397,"{'Ba': 2.0, 'Gd': 3.0, 'Cr': 5.0, 'Fe': 2.0, '...",5,oxide
47,YbGd3F12,mp-1215594,0.000000,-4.1427,-0.2041,"{'Yb': 3.0, 'Gd': 3.0, 'F': -1.0}",3,fluoride
70,Dy11Y5O24,mp-766279,0.000417,-3.9979,NaN,"{'Dy': 3.0, 'Y': 3.0, 'O': -2.0}",3,oxide
74,NaLaF4,mp-1220826,0.000000,-3.9914,-0.0171,"{'Na': 1.0, 'La': 3.0, 'F': -1.0}",3,fluoride
82,NaSmF4,mp-1220915,0.000000,-3.9742,-0.0305,"{'Na': 1.0, 'Sm': 3.0, 'F': -1.0}",3,fluoride
...,...,...,...,...,...,...,...,...
2653,Zn3PBr3,mp-1215629,0.000000,-0.9335,-0.0154,"{'Zn': 2.0, 'P': -3.0, 'Br': -1.0}",3,bromide
2668,Cu2Ag5Se4NO15,mp-1225986,0.000000,-0.9062,-0.0181,"{'Cu': 2.0, 'Ag': 1.0, 'Se': 4.0, 'N': 5.0, 'O...",5,oxide
2826,Zn3P6N12Cl,mp-1215763,0.000000,-0.6356,-0.0394,"{'Zn': 2.0, 'P': 5.0, 'N': -2.9166666666666665...",4,chloride
2922,Co(NF3)4,mp-1214815,0.000000,-0.5108,-0.0028,"{'Co': 2.0, 'N': 2.5, 'F': -1.0}",3,fluoride


#### Final step: manual curation of the final dataset

Google searches of some of these compounds indicate that several have been synthesized very recently and thus would not be captured by any of our resources.

To create the final dataset, one should search the literature for any recent mentions of these compounds.

We performed this manual curation step in Nov 2022; the results are saved as `final_targets_manually_cleaned_11_11_22.xlsx`.